# Data Analysis
## Analysis of the users dataset

In [6]:
from pathlib import Path
from src.data.data_reader import DataReader
import os
import pandas as pd
import re
import json

while str(os.getcwd())[-3:] != 'src':  # Execute from src-directory root
    os.chdir('..')

In [7]:
entries = DataReader._get_entries_from_file(Path('..', 'data', DataReader.EXPECTED_FILES[4]))
RELEVANT_USER_FIELDS = [
    'user_id',
    'name',
    'review_count',  # TODO: Check of dit correct is?
    'friends',
    'useful',
    'funny',
    'cool',
    'fans',
    'compliments'  # Sum of all compliment fields
]

# Combine all compliments
compliment_fields = [
    'compliment_hot',
    'compliment_more',
    'compliment_profile',
    'compliment_cute',
    'compliment_list',
    'compliment_note',
    'compliment_plain',
    'compliment_cool',
    'compliment_funny',
    'compliment_writer',
    'compliment_photos'
]
combined_compliments = DataReader._filter_entries(entries, compliment_fields)
combined_compliments = [sum(x.values()) for x in combined_compliments]
for entry, sum_combined_for_entry in zip(entries, combined_compliments):
    entry['compliments'] = sum_combined_for_entry

filtered_entries = DataReader._filter_entries(entries, RELEVANT_USER_FIELDS)
users = pd.DataFrame.from_records(filtered_entries)
users

,user_id,name,review_count,friends,useful,funny,cool,fans,compliments
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",7217,1259,5994,267,2873
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,"ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",43091,13066,27281,3138,20631
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,"LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",2086,1010,1003,52,585
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,"enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",512,330,299,28,136
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",29,15,7,1,4
...,...,...,...,...,...,...,...,...,...
1987892,fB3jbHi3m0L2KgGOxBv6uw,Jerrold,23,None,7,0,0,0,0
1987893,68czcr4BxJyMQ9cJBm6C7Q,Jane,1,None,0,0,0,0,0
1987894,1x3KMskYxOuJCjRz70xOqQ,Shomari,4,None,1,1,0,0,0
1987895,ulfGl4tdbrH05xKzh5lnog,Susanne,2,None,0,0,0,0,0


In [8]:
users['friends'] = users['friends'].map(lambda friend_str: friend_str.split(', '))


0          [NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8q...
1          [ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0...
2          [LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgy...
3          [enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74V...
4          [PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMb...
                                 ...                        
1987892                                               [None]
1987893                                               [None]
1987894                                               [None]
1987895                                               [None]
1987896                                               [None]
Name: friends, Length: 1987897, dtype: object